In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/IndividualProject/

In [1]:
from datetime import datetime

import os
import glob
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

# Place all your code inside this block
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from sklearn.metrics import recall_score, f1_score, precision_score
import pickle as pkl
import os
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

from ST_GCN.feeder import Feeder
from ST_GCN.st_gcn import Model as ST_GCN
from CTR_GCN.ctrgcn import Model as CTR_GCN

from SkateFormer.feeder_skateformer import Feeder as Feeder_skateformer
from SkateFormer.SkateFormer import SkateFormer_ as SkateFormer

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
def mixup_data(x, y, alpha=0.3):
    """
    Applies Mixup augmentation on skeleton data.

    Args:
        x: Tensor of shape (batch_size, channels, frames, joints, 1)
        y: Tensor of shape (batch_size,)
        alpha: Mixup hyperparameter (default=0.2)

    Returns:
        Mixed input, Mixed target, Lambda value
    """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)  # Sample λ from Beta(α, α)
    else:
        lam = 1

    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)  # Shuffle indices

    mixed_x = lam * x + (1 - lam) * x[index, :]  # Mixup on inputs
    y_a, y_b = y, y[index]  # Keep original labels

    return mixed_x, y_a, y_b, lam


In [3]:
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """
    Compute the Mixup loss as a weighted sum of two labels.
    """
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [4]:
def compute_effective_number_weights(labels, beta=0.9999):
    """
    Compute class weights based on the effective number of samples.

    Args:
        labels (array-like): Array or list of class labels.
        beta (float): Hyperparameter in [0, 1). Typically very close to 1.

    Returns:
        dict: Mapping from class label to computed weight.
    """
    classes, counts = np.unique(labels, return_counts=True)
    weights = {}
    for cls, count in zip(classes, counts):
        effective_num = 1.0 - beta ** count
        weights[cls] = (1.0 - beta) / effective_num if effective_num != 0 else 0.0
    # Optional: normalize weights so that their sum is 1
    weight_sum = sum(weights.values())
    weights = {cls: float(w / weight_sum) for cls, w in weights.items()}
    return weights


In [5]:
class EarlyStopping:
    def __init__(self, patience=2, delta=0.005):
        """
        Args:
            patience (int): How many epochs to wait after last improvement.
            delta (float): Minimum change to qualify as an improvement.
            path (str): Path to save the best model.
        """
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.best_acc = None
        self.best_model = None
        self.epochs_no_improve = 0
        self.early_stop = False

    def __call__(self, val_loss, metric, model):
        """
        Check if the validation loss improved, otherwise update patience counter.
        """
        score = -val_loss  # Lower validation loss is better

        if self.best_score is None or score > self.best_score + self.delta:
            self.best_score = score
            self.epochs_no_improve = 0
            if self.best_acc is None or  metric > self.best_acc:
                self.best_acc = metric
                self.best_model = model
        else:
            self.epochs_no_improve += 1

        if self.epochs_no_improve >= self.patience:
            self.early_stop = True


In [6]:
# Fix Random seed
torch.manual_seed(0)
np.random.seed(0)


In [7]:
def plot_confuse_matrix(y_true, y_pred, classes):
    """
    Plot confusion matrix using seaborn.
    """
    cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(cm, annot=True, cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title('Confusion Matrix')
    plt.show()

In [8]:
def show_topk(result, label, k=1):
    """
    Computes top-k accuracy. By default, k=1 => top-1 accuracy.
    """
    topk_idx = result.argsort()
    hit_top_k = [l in topk_idx[i, -k:] for i, l in enumerate(label)]
    accuracy = sum(hit_top_k) / len(hit_top_k)
    return accuracy * 100

In [9]:
def train(epoch, scheduler, model, optimizer, criterion, data_loader, device, 
          disease="dystonia", model_type='st_gcn', mixup=True, verbose=True):
    model.train()
    print("Learning Rate:", scheduler.get_last_lr()[0])

    loader = data_loader['train']
    loss_value = []

    
    for batch_idx, batch in enumerate(loader):
        if model_type == 'st_gcn' or model_type == 'ctr_gcn':
            data, label = batch
            data = data.float().to(device)
        elif model_type == 'skateformer':
            data, label, index_t = batch
            data = data.float().to(device)
            index_t = index_t.long().to(device)
        else:
            raise ValueError("Invalid model type")
        
        # Using 4th column (index=3) as label, adjust if needed
        if disease == "dystonia_duration":
            label = label[:, 3].long().to(device)
        elif disease == "dystonia_amplitude":
            label = label[:, 4].long().to(device)
        elif disease == "choreoathetosis_duration":
            label = label[:, 5].long().to(device)
        elif disease == "choreoathetosis_amplitude":
            label = label[:, 6].long().to(device)
        elif disease == "dystonia":
            label = label[:, 7].long().to(device)
        elif disease == "choreoathetosis":
            label = label[:, 8].long().to(device)
        else:
            raise ValueError("Invalid disease type")

        mixup_probability = np.random.rand()
        if mixup_probability < 0.3 and mixup:
            mixup1 = True
        else:
            mixup1 = False

        # Mixup
        if mixup1:
            data, label_a, label_b, lam = mixup_data(data, label)
            if model_type == 'st_gcn' or model_type == 'ctr_gcn':
                output = model(data)
            elif model_type == 'skateformer':
                output = model(data, index_t)
            else:
                raise ValueError("Invalid model type")
            
            loss = mixup_criterion(criterion, output, label_a, label_b, lam)
        else:
            if model_type == 'st_gcn' or model_type == 'ctr_gcn':
                output = model(data)
            elif model_type == 'skateformer':
                output = model(data, index_t)
            else:
                raise ValueError("Invalid model type")
            loss = criterion(output, label)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_value.append(loss.item())

        if (batch_idx + 1) % 10 == 0 and verbose:
            print("Epoch: {}/{} || Batch: {}/{} || Loss: {:.4f}".format(
                epoch + 1, EPOCH, batch_idx + 1, len(loader), loss.item()))

    # End-of-epoch logs
    mean_loss = float(np.mean(loss_value))
    if verbose:
        print("Epoch: {}/{} => Mean Loss: {:.4f}".format(epoch + 1, EPOCH, mean_loss))

    return mean_loss

In [10]:
def test(epoch, model, criterion, data_loader, device, evaluation=True, model_type='st_gcn',
         disease="dystonia", verbose=True):
    """
    For validation or test: pass evaluation=True to compute loss & metrics.
    Returns (mean_loss, top1, f1, recall, precision, preds, labels) if evaluation=True.
    Otherwise, just returns the raw outputs.
    """
    model.eval()
    loader = data_loader['val']
    loss_value = []
    result_frag = []
    label_frag = []

    with torch.no_grad():
        for batch_idx, batch in enumerate(loader):
            if model_type == 'st_gcn' or model_type == 'ctr_gcn':
                data, label = batch
                data = data.float().to(device)
            elif model_type == 'skateformer':
                data, label, index_t = batch
                data = data.float().to(device)
                index_t = index_t.long().to(device)
            else:
                raise ValueError("Invalid model type")
            
            if disease == "dystonia_duration":
                label_gpu = label[:, 3].long().to(device)
            elif disease == "dystonia_amplitude":
                label_gpu = label[:, 4].long().to(device)
            elif disease == "choreoathetosis_duration":
                label_gpu = label[:, 5].long().to(device)
            elif disease == "choreoathetosis_amplitude":
                label_gpu = label[:, 6].long().to(device)
            elif disease == "dystonia":
                label_gpu = label[:, 7].long().to(device)
            elif disease == "choreoathetosis":
                label_gpu = label[:, 8].long().to(device)
            else:
                raise ValueError("Invalid disease type")

            if model_type == 'st_gcn' or model_type == 'ctr_gcn':
                output = model(data)
            elif model_type == 'skateformer':
                output = model(data, index_t)
            else:
                raise ValueError("Invalid model type")
            
            result_frag.append(output.cpu().numpy())

            if evaluation:
                loss = criterion(output, label_gpu)
                loss_value.append(loss.item())
                label_frag.append(label_gpu.cpu().numpy())

    result = np.concatenate(result_frag, axis=0)

    if evaluation and len(loss_value) > 0:
        mean_loss = np.mean(loss_value)
        labels = np.concatenate(label_frag, axis=0)
        preds = np.argmax(result, axis=1)

        top1 = show_topk(result, labels, k=1)
        f1 = f1_score(labels, preds, average='macro', zero_division=True)
        recall = recall_score(labels, preds, average='macro', zero_division=True)
        precision = precision_score(labels, preds, average='macro', zero_division=True)
        
        if verbose:
            print("Val Mean Loss: {:.4f}  Top-1: {:.2f}%  F1: {:.2f}%  Recall: {:.2f}%  Precision: {:.2f}%"
                .format(mean_loss, top1, f1 * 100, recall * 100, precision * 100))

        return mean_loss, top1, f1, recall, precision, preds, labels
    else:
        # Either no evaluation or empty loader
        return None

In [22]:
def main(dataset_loader, fold, subject_id, model_type='st_gcn', disease="dystonia", verbose=True):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize model
    if model_type == "st_gcn":
        model = ST_GCN(
            num_class=NUM_CLASS,
            in_channels=IN_CHANNEL,
            graph_args={'strategy': 'spatial'},
            dropout=DROPOUT,
            edge_importance_weighting=True
        ).to(device)
    elif model_type == "ctr_gcn":
        model = CTR_GCN(
            graph_args={'strategy': 'spatial'}, 
            drop_out=DROPOUT, 
            in_channels=IN_CHANNEL, 
            num_class=NUM_CLASS).to(device)
    elif model_type == "skateformer":
        model = SkateFormer(
            in_channels=6,
            num_frames=100,
            num_points=5,
            num_classes=NUM_CLASS,
        )
    else:
        raise ValueError("Invalid model type")

    # Example weighting if you have imbalance
    if disease == "dystonia_duration":
        ld = dataset_loader['train'].dataset.label[:, 3]
    elif disease == "dystonia_amplitude":
        ld = dataset_loader['train'].dataset.label[:, 4]
    elif disease == "choreoathetosis_duration":
        ld = dataset_loader['train'].dataset.label[:, 5]
    elif disease == "choreoathetosis_amplitude":
        ld = dataset_loader['train'].dataset.label[:, 6]
    elif disease == "dystonia":
        ld = dataset_loader['train'].dataset.label[:, 7]
    elif disease == "choreoathetosis":
        ld = dataset_loader['train'].dataset.label[:, 8]
    else:
        raise ValueError("Invalid disease type")
    alpha_dict = compute_effective_number_weights(ld, beta=0.995)  # If you have a custom function
    alpha = list(alpha_dict.values())
    class_weight = torch.tensor(alpha).to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weight)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler1 = CosineAnnealingLR(optimizer, T_max=EPOCH, eta_min=1e-7)
    scheduler2 = StepLR(optimizer, step_size=10, gamma=0.5)

    early_stopping = EarlyStopping(patience=5, delta=0.0001)

    train_losses, val_losses = [], []
    best_model = None
    best_metrics = None  # (val_loss, top1, f1, recall, precision)

    for epoch in range(EPOCH):
        # Train
        if verbose:
            print(f"\n[Subject {subject_id}, Fold {fold}] Epoch {epoch+1}/{EPOCH} - TRAIN")
        train_loss = train(epoch, scheduler1, model, optimizer, criterion, dataset_loader, 
                           device, mixup=True, verbose=verbose, disease=disease, model_type=model_type)
        train_losses.append(train_loss)

        # Validate
        if verbose:
            print(f"[Subject {subject_id}, Fold {fold}] Epoch {epoch+1}/{EPOCH} - VALIDATION")
        val_out = test(epoch, model, criterion, dataset_loader, device, 
                       evaluation=True, verbose=verbose, disease=disease, model_type=model_type)
        if val_out is not None:
            val_loss, top1, f1, recall, precision, preds, labels = val_out
            val_losses.append(val_loss)
        else:
            # If empty val set
            val_loss, top1, f1, recall, precision = (np.inf, 0, 0, 0, 0)
            val_losses.append(val_loss)

        # Step schedulers
        scheduler1.step()
        scheduler2.step()

        # Early stopping check
        metric_to_optimize = 0.6 * f1 + 0.4 * top1
        early_stopping(val_loss, metric_to_optimize, model)
        best_model = early_stopping.best_model

        # Save checkpoint every 5 epochs
        if (epoch + 1) % 5 == 0 and verbose:
            now_str = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")
            ckpt_path = f'./Data/6-leave_one_out/subject{subject_id}/fold_{fold}/{disease}/{model_type}/check_points/checkpoint_epoch_{epoch+1}_{now_str}.pth'
            torch.save(model.state_dict(), ckpt_path)
            print(f"[INFO] Saved checkpoint => {ckpt_path}")

        if early_stopping.early_stop and verbose:
            print("[INFO] Early stopping triggered.")
            break

    # Save the best model for this fold
    best_model_path = f'./Data/6-leave_one_out/subject{subject_id}/fold_{fold}/{disease}/{model_type}/best_model/best_model.pth'
    torch.save(best_model.state_dict(), best_model_path)
    if verbose:
        print(f"[INFO] Best model stored => {best_model_path}")

    # Plot training curves (optional)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.legend()
    plt.title(f"Subject {subject_id} - Fold {fold} Training")
    plt.show()

    return best_model_path  # Return path to best model for later inference

In [23]:
def inference_on_test(best_model_path, test_data_path, test_labels_path, device, model_type='st_gcn', disease="dystonia"):
    """
    Loads the best model, runs inference on test_data, returns predictions & labels.
    """
    if model_type == "st_gcn":
        model = ST_GCN(
            num_class=NUM_CLASS,
            in_channels=IN_CHANNEL,
            graph_args={'strategy': 'spatial'},
            dropout=DROPOUT,
            edge_importance_weighting=True
        ).to(device)
    elif model_type == "ctr_gcn":
        model = CTR_GCN(
            graph_args={'strategy': 'spatial'}, 
            drop_out=DROPOUT, 
            in_channels=IN_CHANNEL, 
            num_class=NUM_CLASS).to(device)
    elif model_type == "skateformer":
        model = SkateFormer(
            num_frames=64
        )
    else:
        raise ValueError(f"Invalid model type: {model_type}")

    # Load weights
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    model.eval()

    # Prepare test dataset
    test_dataset = Feeder(data_path=test_data_path,
                                   label_path=test_labels_path,
                                   mmap=False, augment=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(test_loader):
            data = data.float().to(device)
            if disease == "dystonia_duration":
                label_gpu = label[:, 3].long().to(device)
            elif disease == "dystonia_amplitude":
                label_gpu = label[:, 4].long().to(device)
            elif disease == "choreoathetosis_duration":
                label_gpu = label[:, 5].long().to(device)
            elif disease == "choreoathetosis_amplitude":
                label_gpu = label[:, 6].long().to(device)
            elif disease == "dystonia":
                label_gpu = label[:, 7].long().to(device)
            elif disease == "choreoathetosis":
                label_gpu = label[:, 8].long().to(device)
            else:
                raise ValueError("Invalid disease type")

            output = model(data)
            preds = torch.argmax(output, dim=1)

            all_preds.append(preds.cpu().numpy())
            all_labels.append(label_gpu.cpu().numpy())

    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    return all_preds, all_labels

In [20]:
def run_loso_5_fold_training(type = "st_gcn", disease = "dystonia", verbose=True):
    """
    For each subjectX in ./Data/LOSO:
      1) load subject test_data/test_labels
      2) for each fold_0..4:
         a) load train_data, val_data
         b) train => yields best_model.pth
         c) track which fold had the best validation metric
      3) after checking all 5 folds, use the best fold's model => test inference
      4) accumulate predictions in global lists => confusion matrix at the end
    """
    base_loso_dir = "./Data/6-leave_one_out"
    subject_dirs = sorted(glob.glob(os.path.join(base_loso_dir, "subject*")))

    # Collect predictions/labels from all subjects' test data
    all_preds_global = []
    all_labels_global = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for subj_path in subject_dirs:
        subject_id = os.path.basename(subj_path).replace("subject", "")
        print(f"\n====== LOSO for subject {subject_id} ======")

        # 1) Load this subject's test data
        test_data_path = os.path.join(subj_path, "test_data.npy")
        test_label_path = os.path.join(subj_path, "test_labels.pkl")

        os.makedirs(os.path.join(subj_path, f"{disease}/{type}/"), exist_ok=True)

        if not (os.path.exists(test_data_path) and os.path.exists(test_label_path)):
            print(f"[WARN] Subject {subject_id} missing test data/labels. Skipping.")
            continue

        # 2) For each fold, we do training => track best fold
        best_fold = None
        best_fold_metric = -999
        best_fold_model_path = None
        best_fold_model = None

        for fold_num in range(5):
            fold_dir = os.path.join(subj_path, f"fold_{fold_num}")

            check_points_dir = os.path.join(fold_dir, f"{disease}/{type}/check_points")
            best_models_dir = os.path.join(fold_dir, f"{disease}/{type}/best_model")

            os.makedirs(check_points_dir, exist_ok=True)
            os.makedirs(best_models_dir, exist_ok=True)

            print(check_points_dir, best_models_dir)

            if not os.path.isdir(fold_dir):
                print(f"[WARN] No fold_{fold_num} directory. Skipping.")
                continue

            # --- Load train/val data ---
            train_data_path = os.path.join(fold_dir, "train_data.npy")
            train_label_path = os.path.join(fold_dir, "train_labels.pkl")
            val_data_path = os.path.join(fold_dir, "val_data.npy")
            val_label_path = os.path.join(fold_dir, "val_labels.pkl")

            if not all(os.path.exists(p) for p in [train_data_path, train_label_path, val_data_path, val_label_path]):
                print(f"[WARN] Missing train/val for fold_{fold_num}. Skipping.")
                continue

            # --- Build DataLoaders ---
            if type == "st_gcn" or type == "ctr_gcn":
                train_dataset = Feeder(data_path=train_data_path,
                                    label_path=train_label_path,
                                    mmap=False, augment=True)
                val_dataset   = Feeder(data_path=val_data_path,
                                    label_path=val_label_path,
                                    mmap=False, augment=True)
            elif type == "skateformer":
                train_dataset = Feeder_skateformer(data_path=train_data_path,
                                    label_path=train_label_path,
                                    mmap=False, augment=True)
                val_dataset   = Feeder_skateformer(data_path=val_data_path,
                                    label_path=val_label_path,
                                    mmap=False, augment=True)

            data_loader = {
                'train': DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2),
                'val':   DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
            }

            # --- Train for this fold => returns path to best model
            best_model_path = main(data_loader, fold_num, subject_id, model_type=type, disease=disease, verbose=verbose)

            # --- Validate on the best model ---
            if type == "st_gcn":
                model_check = ST_GCN(
                    num_class=NUM_CLASS,
                    in_channels=IN_CHANNEL,
                    graph_args={'strategy': 'spatial'},
                    dropout=DROPOUT,
                    edge_importance_weighting=True
                ).to(device)
            elif type == "ctr_gcn":
                model_check = CTR_GCN(graph_args={'strategy': 'spatial'}, 
                                      drop_out=DROPOUT, 
                                      in_channels=IN_CHANNEL, 
                                      num_class=NUM_CLASS).to(device)
            elif type == "skateformer":
                model_check = SkateFormer(
                    in_channels=6,
                    num_frames=100,
                    num_points=5,
                    num_classes=NUM_CLASS,
                ).to(device)
            else:
                print(f"[WARN] Unknown model type: {type}. Skipping.")
                return

            model_check.load_state_dict(torch.load(best_model_path, map_location=device))
            model_check.eval()

            val_loss, top1, f1_val, r, p, preds_val, labels_val = test(
                epoch=EPOCH,
                model=model_check,
                criterion=nn.CrossEntropyLoss(),
                data_loader=data_loader,
                device=device,
                evaluation=True,
                verbose=verbose,
                disease=disease
            )

            # Weighted metric (e.g., 0.6*f1 + 0.4*top1)
            fold_metric = 0.6*f1_val + 0.4*top1
            print(f"[Subject {subject_id}] Fold {fold_num} => Weighted metric: {fold_metric:.2f}")

            # Keep track of the best fold
            if fold_metric > best_fold_metric:
                best_fold_metric = fold_metric
                best_fold = fold_num
                best_fold_model_path = best_model_path
                best_fold_model = model_check

        # 3) Inference on test data using the best fold
        if best_fold_model_path is None:
            print(f"[WARN] Subject {subject_id} has no valid folds. Skipping test inference.")
            continue
        
        torch.save(best_fold_model.state_dict(), f"./Data/6-leave_one_out/subject{subject_id}/{disease}/{type}/best_model.pth")
        print(f"\n[Subject {subject_id}] Best fold is {best_fold} with metric {best_fold_metric:.2f}. Inference on test...")
        preds_subj, labels_subj = inference_on_test(best_fold_model_path, test_data_path, test_label_path, device, model_type=type, disease=disease)

        # 4) Store them in global lists
        all_preds_global.append(preds_subj)
        all_labels_global.append(labels_subj)

    # 5) Final confusion matrix / classification report across all subjects
    if len(all_preds_global) == 0:
        print("\nNo predictions collected. Check your data/folder structure.")
        return

    all_preds_global = np.concatenate(all_preds_global, axis=0)
    all_labels_global = np.concatenate(all_labels_global, axis=0)

    print("\n=========== Global LOSO Test Results ===========")
    if disease == "dystonia" or disease == "choreoathetosis":
        classes = [0, 1]
    else:
        classes = [0, 1, 2, 3, 4]
    plot_confuse_matrix(all_labels_global, all_preds_global, classes=classes)
    print("\nClassification Report:\n", classification_report(all_labels_global, all_preds_global))

# Dystonia Duration

## ST-GCN

In [24]:
if __name__ == '__main__':
    LEARNING_RATE = 5e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="st_gcn", disease="dystonia_duration", verbose=False)


====== LOSO for subject 1 ======
./Data/6-leave_one_out/subject1/fold_0/dystonia_duration/st_gcn/check_points ./Data/6-leave_one_out/subject1/fold_0/dystonia_duration/st_gcn/best_model

[Subject 1, Fold 0] Epoch 1/30 - TRAIN
Learning Rate: 0.0005
[Subject 1, Fold 0] Epoch 1/30 - VALIDATION

[Subject 1, Fold 0] Epoch 2/30 - TRAIN
Learning Rate: 0.0004986307477473
[Subject 1, Fold 0] Epoch 2/30 - VALIDATION

[Subject 1, Fold 0] Epoch 3/30 - TRAIN
Learning Rate: 0.0004945379928034148
[Subject 1, Fold 0] Epoch 3/30 - VALIDATION

[Subject 1, Fold 0] Epoch 4/30 - TRAIN
Learning Rate: 0.00048776657624797375
[Subject 1, Fold 0] Epoch 4/30 - VALIDATION

[Subject 1, Fold 0] Epoch 5/30 - TRAIN
Learning Rate: 0.00047839068713776816
[Subject 1, Fold 0] Epoch 5/30 - VALIDATION
[INFO] Saved checkpoint => ./Data/6-leave_one_out/subject1/fold_0/dystonia_duration/st_gcn/check_points/checkpoint_epoch_5_19-03-2025_04:19:45.pth

[Subject 1, Fold 0] Epoch 6/30 - TRAIN
Learning Rate: 0.0004665130496759206
[

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
_pickle.UnpicklingError: pickle data was truncated


KeyboardInterrupt: 

## CTR-GCN

In [16]:
if __name__ == '__main__':
    LEARNING_RATE = 5e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="ctr_gcn", disease="dystonia_duration", verbose=False)


====== LOSO for subject 1 ======
./Data/6-leave_one_out/subject1/fold_0/dystonia_duration/ctr_gcn/check_points ./Data/6-leave_one_out/subject1/fold_0/dystonia_duration/ctr_gcn/best_model

[Subject 1, Fold 0] Epoch 1/30 - TRAIN
Learning Rate: 0.0005


KeyboardInterrupt: 

## SkateFormer

In [21]:
if __name__ == '__main__':
    LEARNING_RATE = 5e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="skateformer", disease="dystonia_amplitude", verbose=True)


====== LOSO for subject 1 ======
./Data/6-leave_one_out/subject1/fold_0/dystonia_amplitude/skateformer/check_points ./Data/6-leave_one_out/subject1/fold_0/dystonia_amplitude/skateformer/best_model


TypeError: SkateFormer.SkateFormer.SkateFormer() got multiple values for keyword argument 'depths'

# Dystonia Amplitude

## ST-GCN

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 5e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="st_gcn", disease="dystonia_amplitude", verbose=False)

## CTR-GCN

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 5e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="ctr_gcn", disease="dystonia_amplitude", verbose=False)

## SkateFormer

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 5e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="skateformer", disease="dystonia_amplitude", verbose=False)

# Choreoaethetosis Duration

## ST-GCN

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 1e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="st_gcn", disease="choreoathetosis_duration", verbose=False)

## CTR-GCN

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 1e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="ctr_gcn", disease="choreoathetosis_duration", verbose=False)

## SkateFormer

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 1e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="skateformer", disease="choreoathetosis_duration", verbose=False)

# Choreoaethetosis Amplitude

## ST-GCN

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 1e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="st_gcn", disease="choreoathetosis_amplitude", verbose=False)

## CTR-GCN

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 1e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="ctr_gcn", disease="choreoathetosis_amplitude", verbose=False)

## SkateFormer

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 1e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 5
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="skateformer", disease="choreoathetosis_amplitude", verbose=False)

# Dystonia

## ST-GCN

In [17]:
if __name__ == '__main__':
    LEARNING_RATE = 5e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 2
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="st_gcn", disease="dystonia", verbose=False)




No predictions collected. Check your data/folder structure.


## CTR-GCN

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 5e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 2
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="ctr_gcn", disease="dystonia", verbose=False)

## SkateFormer

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 5e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 2
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="skateformer", disease="dystonia", verbose=False)

# Choreoathetosis

## ST-GCN

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 1e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 2
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="st_gcn", disease="choreoathetosis", verbose=False)

## CTR-GCN

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 1e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 2
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="ctr_gcn", disease="choreoathetosis", verbose=False)

## SkateFormer

In [ ]:
if __name__ == '__main__':
    LEARNING_RATE = 1e-4
    EPOCH = 30
    BATCH_SIZE = 32
    WEIGHT_DECAY = 1e-4
    DROPOUT = 0.5

    NUM_CLASS = 2
    IN_CHANNEL = 6
    run_loso_5_fold_training(type="skateformer", disease="choreoathetosis", verbose=False)